In [1]:
# all imports

import numpy as np

import sys
sys.path.append('.')

from sfepy.base.base import IndexedStruct, Struct
from sfepy.discrete import (FieldVariable, Material, Integral, Integrals,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson

In [2]:
# this is the postprocessing function from sfepy.examples.linear_elasticity.its2D_2
def stress_strain(out, pb, state):
    """
    Calculate and output strain and stress for given displacements.
    """
    from sfepy.base.base import Struct

    ev = pb.evaluate
    strain = ev('ev_cauchy_strain.2.Surface(u)', mode='el_avg')
    stress = ev('ev_cauchy_stress.2.Surface(Aluminum.D, u)', mode='el_avg',
                copy_materials=False)

    out['cauchy_strain'] = Struct(name='output_data', mode='cell',
                                  data=strain, dofs=None)
    out['cauchy_stress'] = Struct(name='output_data', mode='cell',
                                  data=stress, dofs=None)

    return out


In [3]:
def nodal_stress(pb, integrals=None):
    '''
    Calculate and print stresses at nodal points.
    '''
    # Update problem time
    pb.time_update()

    if integrals is None:
        integrals = pb.get_integrals()

    # Evaluate stress at quadrature points
    stress = pb.evaluate('ev_cauchy_stress.ivn.Surface(Aluminum.D, u)', mode='qp', 
                         integrals=integrals, copy_materials=False)
    
    # Create stress field variable from quadrature points
    sfield = Field.from_args('stress', np.float64, (3,), 
                             pb.domain.regions['Surface'])
    svar = FieldVariable('sigma', 'parameter', sfield, 
                         primary_var_name='(set-to-None)')
    svar.set_from_qp(stress, integrals['ivn'])

    return svar()

In [4]:
"""
Testing a triangle under load
"""

# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum

# Define forces in Newtons
fy = 6 * 1e3  # Example force in the y-direction (kn)
force_ratio = np.tan(np.radians(10))  # Work with an angle of 10 degrees.
fx = fy * force_ratio  # Horizontal force

mesh = Mesh.from_file('triangle2d.mesh')
domain = FEDomain('domain', mesh)

# Get the bounding box of the domain to determine the boundary values
# min_x, max_x = domain.get_mesh_bounding_box()[:, 0]
# min_y, max_y = domain.get_mesh_bounding_box()[:, 1]

# print(min_x, max_x)
# print(min_y, max_y)

# # Define a small epsilon value for numerical tolerance
# eps = 1e-8 * (max_x - min_x)

#set up entire region as Omega
Surface = domain.create_region('Surface', 'all')

# Create regions for the top and bottom vertices at  (0, 1), (0,0) and (1,0)
Top = domain.create_region('Top', 'vertex 3', 'vertex')
Bottom_Left = domain.create_region('Bottom_Left', 'vertex 1', 'vertex')                                        
Bottom_Right = domain.create_region('Bottom_Right', 'vertex 2', 'vertex')

field = Field.from_args('fu', np.float64, 'vector', Surface, approx_order=2)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

# define elastic properties:
# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum
aluminum = Material('Aluminum', D=stiffness_from_youngpoisson(2, young=E, poisson=nu))

# # Define the force material load (force vector in the x,y-directions)
load = Material('Load', values={'.val' : [fx, fy]})

# Use order 2 integrals
integral2 = Integral('i', order=2)
integral0 = Integral('i', order=0)

# Set up linear elastic conditions 
t_elastic = Term.new('dw_lin_elastic(Aluminum.D, v, u)',
                integral2, Surface, Aluminum=aluminum, v=v, u=u)

# Apply the force at the top vertex using a volume load
t_load = Term.new('dw_point_load(Load.val, v)', integral0, Top, Load=load, v=v)

eq = Equation('balance', t_elastic + t_load)
eqs = Equations([eq])

# Apply boundary conditions to the two bottom vertices
fix_bottom_left = EssentialBC('fix_bottom_left', Bottom_Left, {'u.all': 0.0})  # Fix both x and y displacements
fix_bottom_right = EssentialBC('fix_bottom_right', Bottom_Right, {'u.all': 0.0})  # Fix both x and y displacements

ls = ScipyDirect({})

nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)



sfepy: reading mesh (triangle2d.mesh)...
sfepy:   number of vertices: 279
sfepy:   number of cells:
sfepy:     2_3: 487
sfepy: ...done in 0.00 s


In [5]:
pb = Problem('triangle load', equations=eqs)

pb.save_regions_as_groups('regions')

# Add the boundary conditions to the problem setup
pb.set_bcs(ebcs=Conditions([fix_bottom_left, fix_bottom_right]))

# Set up and solve the problem
pb.set_solver(nls)

status = IndexedStruct()


sfepy: saving regions as groups...
sfepy:   Surface
sfepy:   Top
sfepy:   Bottom_Left
sfepy:   Bottom_Right
sfepy: ...done


In [6]:
variables = pb.solve(status=status)

print('Nonlinear solver status:\n', nls_status)
print('Stationary solver status:\n', status)

#pb.save_state('triangle_load.vtk', variables)

sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (2084, 2084)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 45824 (1.06e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     Load
sfepy:     Aluminum
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 6.092560e+03 (rel: 1.000000e+00)
sfepy:   residual:    0.01 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: warning: linear system solution precision is lower then the value set in solver options! (err = 1.974852e-09 < 1.000000e-10)
sfepy: nls: iter: 1, residual: 1.998187e-09 (rel: 3.279716e-13)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: solved in 1 steps in 0.03 seconds
Nonlinear solver status:
 IndexedStruct
  condition:
    1
  err:
    1.9981868000187418e-09
  err0:
    6092.5596713144705
  ls_n_iter:
    -1
  n_iter:
    1
  time:
    0.021584250003797933
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
Stationary solver status:
 IndexedStruct
  n_step:
    1
  time:
    0.02864400000544265


In [7]:
ev = pb.evaluate

strain = ev('ev_cauchy_strain.2.Surface(u)', mode='el_avg')
#stress = ev('ev_cauchy_stress.2.Surface(Aluminum.D, u)', mode='el_avg',
#                copy_materials=False)

sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Surface(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s


In [8]:
# post processing for stress and stress and nodes:
from sfepy.discrete.fem.geometry_element import geometry_data

out = variables.create_output()
out = stress_strain(out, pb, variables)
pb.save_state('triangle_load_ext.vtk', out=out)

gdata = geometry_data['2_3']
nc = len(gdata.coors)

integral_vn = Integral('ivn', coors=gdata.coors,
                        weights=[gdata.volume / nc] * nc)

sdata = nodal_stress(pb, integrals=Integrals([integral_vn])) # get the stress data at nodes

# Point load values
mat = pb.get_materials()['Load']
L_x, L_y = mat.get_data('special', 'val')

# Example node indices to print stresses for
specific_nodes = [0, 1, 2]  # Replace these with actual node indices you're interested in

for node in specific_nodes:
    # Horizontal tensile stress (sigma_xx)
    sigma_xx = sdata[node, 0]

    # Vertical compressive stress (sigma_yy)
    sigma_yy = sdata[node, 1]

    # Print stress at the node
    print(f'Node {node}: Horizontal stress (sigma_xx) = {sigma_xx:.5e} Pascals')
    print(f'Node {node}: Vertical stress (sigma_yy) = {sigma_yy:.5e} Pascals')


sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Surface(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.2.Surface(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: updating variables...
sfepy: ...done
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.ivn.Surface(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [9]:
# integral2 = Integral('i1', order=2)  # or the appropriate order for your problem
# stress = pb.evaluate('ev_cauchy_stress.2.Omega(m.D, u)', mode='el_avg', m = m, integral=integral2)

# # Create an output structure for the stress data at nodes
# stress_out = {
#     'cauchy_stress': Struct(name='stress_data', mode='el_avh', data=stress, dofs=None)
# }

In [10]:
stress_out['cauchy_stress'].data.shape

NameError: name 'stress_out' is not defined

In [ ]:
qp_coors, qp_weights = integral.get_qp('2_3')  # '2_3' for 2D triangles

In [ ]:
qp_coors

array([[0.16666667, 0.16666667],
       [0.66666667, 0.16666667],
       [0.16666667, 0.66666667]])

In [ ]:
displacement_out = variables.create_output()

In [ ]:
displacement_out['u'].data.shape

(279, 2)

In [ ]:
stress_out['cauchy_stress'].data.shape

(487, 1, 3, 1)

In [ ]:
pb.save_state('displacement_out.vtk', out=displacement_out)

Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

In [ ]:
pb.save_state('stress_output.vtk', out=output_data)


In [ ]:
stress

array([[[[ 19.25732568],
         [ 19.74388076],
         [-21.34047028]]],


       [[[  0.        ],
         [  0.        ],
         [  0.        ]]],


       [[[ 37.06850137],
         [ 46.03084566],
         [-41.42226812]]],


       ...,


       [[[ 11.17687793],
         [-22.82142084],
         [ -4.28716581]]],


       [[[ 17.45381033],
         [ 27.15767556],
         [-21.79927093]]],


       [[[ -7.03076706],
         [  2.18073572],
         [ 11.96223269]]]])

In [ ]:
# Compute the normal stress components
sigma_xx = stress[:, 0, 0, 0]  # σ_xx component
sigma_yy = stress[:, 0, 1, 0]  # σ_yy component

# Prepare output for visualization
stress_normal_out = {
    'sigma_xx': Struct(name='output_data', mode='cell', data=sigma_xx, dofs=None),
    'sigma_yy': Struct(name='output_data', mode='cell', data=sigma_yy, dofs=None),
}

# Save the normal stress components for visualization
#pb.save_state('normal_stress_output.vtk', out=stress_normal_out)

In [ ]:
print(stress.shape)

(487, 1, 3, 1)


In [ ]:
sigma_xx

array([19.25732568,  0.        , 37.06850137, ..., 11.17687793,
       17.45381033, -7.03076706])

In [ ]:
u_data = variables['u'].data  # Displacement data
u_data[0].shape

(2088,)

In [ ]:
stress.shape

(487, 1, 3, 1)

In [ ]:
sigma_xx

array([19.25732568,  0.        , 37.06850137, ..., 11.17687793,
       17.45381033, -7.03076706])

In [ ]:
variables

Variables

In [ ]:
print(dir(variables))

['__add__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_create_dof_info', '_format_sequence', '_objs', '_str', 'adi', 'adof_conns', 'advance', 'append', 'apply_ebc', 'apply_ic', 'as_dict', 'avdi', 'bc_of_vars', 'check_vec_size', 'copy', 'create_output', 'create_reduced_vec', 'create_vec', 'di', 'dtype', 'ebcs', 'epbcs', 'equation_mapping', 'extend', 'fill_state', 'from_conf', 'get', 'get_dual_names', 'get_indx', 'get_lcbc_operator', 'get_matrix_shape', 'get_names', 'get_reduced_state', 'get_state', 'get_state_parts', 'get_vec_part', 'has_ebc', 'has_eq_map', 'has_key', 'has_lcbc', '

In [ ]:
# Get quadrature point coordinates for the element type you're using (e.g., '2_3' for triangles)
qp_coors, qp_weights = pb.get_qp('2_3')

AttributeError: 'Problem' object has no attribute 'get_qp'